# 🏗️ Iteracja 2: Structured Output (Pydantic + Instructor)

**Czas:** ~45 minut  
**Poziom:** Podstawowy

---

## Cel iteracji

Zastąpić kruche parsowanie surowych stringów **gwarantowanym, walidowanym outputem** przy użyciu Pydantic i Instructor.

---

## Problem z Iteracji 1

W poprzedniej iteracji model zwracał surowy tekst:

```
"bug, performance"  ✅ Działa
"Bug and Performance"  ❌ Różna wielkość liter
"The review mentions bugs and performance issues."  ❌ Zdanie zamiast etykiet
"[bug, performance]"  ❌ Nawiasy kwadratowe
```

Parsowanie stringów jest **kruche** - każda zmiana formatu psuje kod.

---

## Rozwiązanie: Pydantic + Instructor

### Co to Pydantic?

**Pydantic** to biblioteka do walidacji danych w Pythonie. Definiujesz model danych jako klasę Python, a Pydantic automatycznie:
- Sprawdza typy danych
- Waliduje wartości (np. czy liczba jest w zakresie 0-1)
- Konwertuje typy gdy to możliwe (string → int)
- Wyrzuca czytelne błędy gdy walidacja się nie powiędzie

```python
from pydantic import BaseModel, Field

class ReviewClassification(BaseModel):
    categories: list[str] = Field(description="Lista kategorii tematycznych")
    confidence: float = Field(ge=0, le=1, description="Pewność klasyfikacji 0-1")
```

### Co to Instructor?

**Instructor** to nakładka na klientów LLM, która:
1. Dodaje instrukcje do promptu mówiące modelowi, żeby zwrócił JSON
2. Parsuje JSON z odpowiedzi
3. Waliduje JSON przez Pydantic
4. Jeśli walidacja się nie powiędzie - automatycznie pyta model ponownie z informacją o błędzie

```python
import instructor
from openai import OpenAI

client = instructor.patch(OpenAI(), mode=instructor.Mode.MD_JSON)

# Zamiast response.choices[0].message.content:
result = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[...],
    response_model=ReviewClassification,  # ← to jest nowe!
)
# result jest teraz obiektem ReviewClassification, nie stringiem!
```

## ⚙️ Setup - środowisko

Ta komórka wykrywa czy jesteś w **Google Colab** czy lokalnie, a następnie:
1. Klonuje repozytorium z GitHub (tylko Colab)
2. Instaluje wymagane biblioteki
3. Konfiguruje ścieżki importów

In [ ]:
import sys
import subprocess
from pathlib import Path

# Wykryj środowisko
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"{'🟡 Google Colab' if IN_COLAB else '💻 Lokalne środowisko'}")

# ============================================================
# KONFIGURACJA REPO (zmień URL na właściwe przed warsztatami)
# ============================================================
REPO_URL = "https://github.com/JSerek/techland-genai-workshop.git"
REPO_DIR = Path("/content/szkolenie_techland") if IN_COLAB else Path(".").resolve().parent
# ============================================================

if IN_COLAB:
    if not REPO_DIR.exists():
        print(f"📥 Klonuję repo...")
        subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
        print("✅ Repo sklonowane")
    else:
        print("🔄 Aktualizuję repo (git pull)...")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)

    # Dodaj repo do sys.path
    sys.path.insert(0, str(REPO_DIR))

    # Zainstaluj zależności
    print("📦 Instaluję biblioteki...")
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", str(REPO_DIR / "requirements.txt"), "-q"],
        check=True
    )
    print("✅ Biblioteki zainstalowane")
else:
    # Lokalnie: dodaj root projektu do sys.path
    sys.path.insert(0, str(REPO_DIR))
    print(f"📂 Ścieżka projektu: {REPO_DIR}")

print("\n✅ Setup zakończony")

In [ ]:
# ============================================================
# 🔑 Konfiguracja API
# W Google Colab: dodaj sekrety w menu po lewej (🔑 ikona)
#   Nazwy sekretów: VERTEX_AI_API_KEY, VERTEX_AI_BASE_URL, MODEL_NAME
#
# Lokalnie: ustaw zmienne środowiskowe lub wpisz wartości bezpośrednio
# ============================================================

if IN_COLAB:
    from google.colab import userdata
    API_KEY   = userdata.get("VERTEX_AI_API_KEY")
    BASE_URL  = userdata.get("VERTEX_AI_BASE_URL")
    MODEL_NAME = userdata.get("MODEL_NAME") or "google/gemini-2.5-flash-lite"
else:
    import os
    API_KEY   = os.environ.get("VERTEX_AI_API_KEY", "TODO: wklej API key")
    BASE_URL  = os.environ.get("VERTEX_AI_BASE_URL", "TODO: wklej endpoint URL")
    MODEL_NAME = os.environ.get("MODEL_NAME", "google/gemini-2.5-flash-lite")

# Walidacja
if not API_KEY or API_KEY == "TODO: wklej API key":
    print("❌ Brak API_KEY! Dodaj sekret 'VERTEX_AI_API_KEY' w Colab Secrets.")
elif not BASE_URL or BASE_URL == "TODO: wklej endpoint URL":
    print("❌ Brak BASE_URL! Dodaj sekret 'VERTEX_AI_BASE_URL' w Colab Secrets.")
else:
    print(f"✅ API skonfigurowane: {MODEL_NAME}")
    print(f"   Endpoint: {BASE_URL[:60]}...")

In [ ]:
# Konfiguracja klienta LLM - dane pobrane z Secrets w komórce powyżej
from src.utils.llm_client import create_client, LLMProvider

PROVIDER = LLMProvider.VERTEX_AI  # Zmień jeśli używasz innego providera

client = create_client(
    provider=PROVIDER,
    api_key=API_KEY,
    base_url=BASE_URL,
)
print(f"✅ Klient LLM gotowy: {PROVIDER.value} / {MODEL_NAME}")

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Ścieżki do danych (względem root repo)
DATA_DIR = REPO_DIR / "data"

# Próbka recenzji
df = pd.read_csv(DATA_DIR / "processed" / "workshop_sample.csv")
review_texts = df["review_text"].tolist()
print(f"✅ Załadowano {len(review_texts)} recenzji")

# Golden dataset
golden_path = DATA_DIR / "evaluation" / "golden_dataset.json"
if golden_path.exists():
    with open(golden_path) as f:
        golden_data = json.load(f)
    golden_texts = [item["review_text"] for item in golden_data]
    golden_labels = [item["labels"] for item in golden_data]
    print(f"✅ Golden dataset: {len(golden_data)} przykładów")
else:
    golden_texts, golden_labels = [], []
    print("⚠️  Golden dataset nie znaleziony - ewaluacja niedostępna")

## 🔍 Porównanie: Raw Output vs Structured Output

Zobaczymy różnicę między podejściem z Iteracji 1 a Structured Output.

In [ ]:
# --- Podejście z Iteracji 1 (raw string) ---
def classify_raw(review_text: str) -> str:
    """Zwraca surowy string - trudny do parsowania."""
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "Klasyfikuj recenzję. Zwróć kategorie oddzielone przecinkami."},
            {"role": "user", "content": review_text},
        ],
        temperature=0.0,
    )
    return response.choices[0].message.content


test_review = review_texts[0]
raw_result = classify_raw(test_review)
print(f"Raw output typ: {type(raw_result).__name__}")
print(f"Raw output wartość: '{raw_result}'")
print(f"\nCzy mogę użyć .categories? {hasattr(raw_result, 'categories')}")

## 🏗️ Definicja Pydantic Model

Definiujemy strukturę, którą chcemy dostać z modelu.

In [ ]:
# ============================================================
# TODO: Uzupełnij kategorie po analizie danych
# ============================================================
CATEGORIES = [
    # "performance", "bug", "story", "gameplay", "graphics", "audio", "price", "other"
]
# ============================================================


class ReviewClassification(BaseModel):
    """Wynik klasyfikacji tematycznej recenzji gry."""

    categories: list[str] = Field(
        description=(
            f"Lista kategorii tematycznych z recenzji. "
            f"Dozwolone wartości: {CATEGORIES}. "
            "Może być jedna lub więcej kategorii."
        )
    )

print("✅ Model ReviewClassification zdefiniowany")
print(f"Pola: {list(ReviewClassification.model_fields.keys())}")

In [ ]:
# --- Podejście z Instructor (structured output) ---
def classify_structured(review_text: str) -> ReviewClassification:
    """Zwraca walidowany obiekt Pydantic - zawsze poprawnej struktury."""
    categories_str = ", ".join(f'"{c}"' for c in CATEGORIES)

    return client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {
                "role": "system",
                "content": f"""Klasyfikuj recenzję gry do kategorii tematycznych.
Dostępne kategorie: {categories_str}
Przypisz jedną lub więcej kategorii pasujących do treści recenzji."""
            },
            {
                "role": "user",
                "content": f"Recenzja: {review_text}"
            },
        ],
        response_model=ReviewClassification,  # ← Instructor używa tego!
        temperature=0.0,
    )


structured_result = classify_structured(test_review)
print(f"Structured output typ: {type(structured_result).__name__}")
print(f"Wartość: {structured_result}")
print(f"\nCzy mogę użyć .categories? {hasattr(structured_result, 'categories')}")
print(f"Kategorie: {structured_result.categories}")

## 🎯 Ćwiczenie: Rozszerz model o dodatkowe pola

### Zadanie:
Dodaj do modelu `ReviewClassification` pola `confidence` i `reasoning`. Sprawdź jak wpływają na jakość klasyfikacji.

> 💡 **Dlaczego `reasoning` przed `categories`?**  
> Zmuszanie modelu do najpierw wypisania rozumowania, a potem decyzji, to prosty sposób na **Chain-of-Thought** - model "myśli" zanim odpowie. Poznamy to dokładniej w Iteracji 3.

In [ ]:
# ============================================================
# 🖊️  ĆWICZENIE: Rozszerz model
# Dodaj pole 'reasoning' (str) i 'confidence' (float 0-1)
# WAŻNE: reasoning powinno być PRZED categories
# ============================================================

class ReviewClassificationV2(BaseModel):
    """Rozszerzona klasyfikacja z reasoning i confidence."""

    # TODO: Dodaj pole 'reasoning' - opis co skłoniło do takiej klasyfikacji
    # reasoning: str = Field(description="...")

    categories: list[str] = Field(
        description=f"Lista kategorii. Dozwolone: {CATEGORIES}"
    )

    # TODO: Dodaj pole 'confidence' - pewność klasyfikacji 0-1
    # confidence: float = Field(ge=0, le=1, description="...")


# Przetestuj rozszerzony model:
# result_v2 = client.chat.completions.create(
#     model=MODEL_NAME,
#     messages=[...],
#     response_model=ReviewClassificationV2,
#     temperature=0.0,
# )
# print(result_v2)

print("Zdefiniuj ReviewClassificationV2 i przetestuj!")

## 📊 Klasyfikacja i ewaluacja

In [ ]:
from tqdm.notebook import tqdm

texts_to_classify = golden_texts if golden_texts else review_texts[:10]

print(f"Klasyfikuję {len(texts_to_classify)} recenzji (structured output)...")

predictions = []
for text in tqdm(texts_to_classify):
    try:
        result = classify_structured(text)
        predictions.append(result.categories)
    except Exception as e:
        print(f"Błąd: {e}")
        predictions.append([])

print("\n✅ Gotowe")

In [ ]:
if golden_labels:
    trial2 = evaluate_trial(
        trial_name=f"{MODEL_NAME} + structured output",
        model=MODEL_NAME,
        prompt_variant="structured output",
        predictions=predictions,
        expected=golden_labels,
        review_texts=golden_texts,
        strategy=MatchStrategy.CONTAINS_ALL,
    )
    trial2.display()
    print(f"\n💾 Accuracy Iteracja 2: {trial2.summary.accuracy:.1%}")

---

## ✅ Podsumowanie Iteracji 2

**Czego się nauczyłeś:**
- Definiowanie modeli danych z Pydantic (`BaseModel`, `Field`)
- Używanie Instructor do gwarantowanego structured output
- Różnica między surowym stringiem a walidowanym obiektem
- Jak `reasoning` pole pomaga modelowi "myśleć" (preview CoT)

**Kluczowa zasada:**  
> Zawsze używaj Pydantic + Instructor w production-grade kodzie. Nigdy nie parsuj stringów z LLM ręcznie.

**Następna iteracja:**  
Explicitnymi instrukcjami Chain-of-Thought jeszcze bardziej poprawimy jakość klasyfikacji.